# Pre-prossessing

## Import des librairies nécessaires

In [1]:
#!pip install nltk numpy matplotlib
#!pip3 install simplenlg
#!pip install deep-translator
#!pip install CoreNLG

In [2]:
from collections import defaultdict
import os

import matplotlib.pyplot as plt
import numpy as np
import random

from deep_translator import GoogleTranslator

from simplenlg.framework import *
from simplenlg.lexicon import *
from simplenlg.realiser.english import *
from simplenlg.phrasespec import *
from simplenlg.features import *

from nltk.parse.generate import generate, demo_grammar
from nltk import CFG

from CoreNLG.NlgTools import NlgTools
from CoreNLG.PredefObjects import TextVar


## Définition des fonctions

In [3]:
def sortbyyear(elem):
     return elem.get('year')      

def search_trans(word, lang):
    for item in TPratiques :
        if item['EN'].lower().strip()==word.lower().strip() :
            return item[lang]
    print("Pas de traduction pour " + word)
    return GoogleTranslator(source='auto', target=lang.lower()).translate(word)

def search_univ(ecole,lang):
    for item in TUniversities :
        if ecole.lower().strip() in item['recherche'].lower().strip() :
            return item[lang]
    print("Pas de traduction pour " + ecole)
    return GoogleTranslator(source='auto', target=lang.lower()).translate(ecole)



def prop_det(prop, det):
    determ=det.strip().lower()
    propo=prop.strip().lower()
    result=''
    if propo=='en' :
        result=propo    
    elif propo=='sur' or propo=='dans' :
        result=propo + " " + determ
    elif determ=='le' :
        result=propo
    elif propo=='du' or propo=='de' :
        if determ=='l\'' :
            result='de l\''
        elif determ=='la':
            result='de la'
        elif determ=='les':
            result='des'
    elif propo=='au' :
        if determ=='l\'' :
            result='à l\''
        elif determ=='la':
            result='à la'
        elif determ=='les':
            result='aux'
    return result

def voyelle(lettre) :
    if lettre in ['a', 'e', 'i', 'o', 'u', 'y', 'h']:
        return True
    else :
        return False

## Chargement des listes de référence (université, pratiques de droits)

In [4]:
elems="" 

list_countries_EU = ('Suisse','Austria','Belgium','Bulgaria','Croatia','Cyprus','Czech Republic','Denmark','Estonia','Finland','France','Germany','Greece','Hungary','Ireland','Italy','Latvia','Lithuania','Luxembourg','Malta','Netherlands','Poland','Portugal','Romania','Slovakia','Slovenia','Spain','Sweden')
    
##chemins vers les fichiers 
path_pratiques="data/pratiques.txt" ##liste des Pratiques de droits dans les 3 langues
path_univ="data/universities.txt" ##liste des dénominations d'université dans les 3 langues

pratiques=open(path_pratiques,'r')
universities=open(path_univ,'r')

##mise en table des pratiques de droit
TPratiques=[{'EN' : "EN",'FR' : "FR",'NL' : "NL"}]
elems=""
for lines in pratiques.readlines():
    elems=lines.split(";")
    TPratiques.append({'EN' : elems[0].strip(),'FR' : elems[1].strip(),'NL' : elems[2].strip()}) 

##mise en table des dénominations d'université
TUniversities=[{'recherche' : "recherche",'EN' : "EN",'FR' : "FR",'NL' : "NL"}]
elems=""
for lines in universities.readlines():
    elems=lines.split(";")
    TUniversities.append({'recherche' : elems[0].strip(),'EN' : elems[1].strip(),'FR' : elems[2].strip(),'NL' : elems[3].strip()}) 

print("la liste de référence des pratiques contient :")
print(TPratiques[:5])
print("************************************************")
print("la liste de référence des universités contient :")
TUniversities[:5]

la liste de référence des pratiques contient :
[{'EN': 'EN', 'FR': 'FR', 'NL': 'NL'}, {'EN': 'Capital Markets', 'FR': 'les Marchés financiers', 'NL': 'Kapitaalmarkten'}, {'EN': 'Civil Law Notaries', 'FR': 'le droit notarial', 'NL': 'Notarissen'}, {'EN': 'Compliance', 'FR': 'la Conformité', 'NL': 'Compliance'}, {'EN': 'Sanctions', 'FR': 'les sanctions', 'NL': 'sancties'}]
************************************************
la liste de référence des universités contient :


[{'recherche': 'recherche', 'EN': 'EN', 'FR': 'FR', 'NL': 'NL'},
 {'recherche': 'University of Lapland',
  'EN': 'University of Lapland',
  'FR': "l'Université de Laponie",
  'NL': 'Universiteit van Lapland'},
 {'recherche': 'Université de Laponie',
  'EN': 'University of Lapland',
  'FR': "l'Université de Laponie",
  'NL': 'Universiteit van Lapland'},
 {'recherche': 'Universiteit van Lapland',
  'EN': 'University of Lapland',
  'FR': "l'Université de Laponie",
  'NL': 'Universiteit van Lapland'},
 {'recherche': 'Katholieke Universiteit Leuven',
  'EN': '<i>Katholieke Universiteit Leuven</i>',
  'FR': 'la <i>Katholieke Universiteit Leuven</i>',
  'NL': 'Katholieke Universiteit Leuven'}]

## Chargement des données relatives à l'avocat contenu dans le fichier txt

In [5]:
# Nom et strucutures des variables
#--------------------------------
#FirstName
#LastName
#Gender : F ou M
#Birthyear yyyy
#PG : PG1;PG2
#Expertise : Expertise1;Expertise2
#TBar : [year;city]
#TDegree : [degree;branch;specialisation;university;country;year]
#TExch=[type;university,country,year]
#Arrival : year yyyy

elems="" 
    
##fichier contenant les informations sur l'avocat
count = 0
pers="Amelie_Poulain" 
path="data/"+pers+".txt" 
 
Info = open(path, 'r')

while True:
    line = Info.readline()
    if not line:
        break
    elems = line.split(":")
    if elems[0].strip() == 'FirstN':
        FirstName= elems[1].strip()
    elif elems[0].strip() == 'LastN':
        LastName= elems[1].strip()
    elif elems[0].strip() == 'Gender':
        Gender= elems[1].strip()
    elif elems[0].strip() == 'Birthyear':
        Birthyear= elems[1].strip()
    elif elems[0].strip() == 'PG':
        PG = elems[1].strip()
    elif elems[0].strip() == 'Expertise':
        Expertise= elems[1].strip()
    elif elems[0].strip() == 'Bar':
        Bar= elems[1].strip()
    elif elems[0].strip() == 'Arrival':
        Arrival = elems[1].strip()
    elif elems[0].strip() == 'Degree':
        Degree = elems[1].strip()
    elif elems[0].strip() == 'Exchange_programme':
        Exchange_programme= elems[1].strip()   
Info.close()

##informations relatives à l'admission au barreau
elems=""
TBar=[]
if len(Bar) > 0 :
    try:
        var=Bar
        var1 = var[1:var.find(']')]
        var = var[var.find(']')+1:]
        elems = var1.split(";")
        TBar=[{'year': int(elems[0].strip()),'city': elems[1].strip()}]        
        while var.startswith('[') :       
            var1 = var[1:var.find(']')]
            var = var[var.find(']')+1:]
            elems = var1.split(";")
            TBar.append({'year': int(elems[0].strip()),'city': elems[1].strip()})        
    except ValueError as ve:
        print('value error : ', Bar)
        print('>', ve)
        
##informations relative à l'admission aux diplomes
elems=""   
TDegree=[]
print(Degree)
if len(Degree) > 0 :
    try:
        var=Degree
        var1 = var[1:var.find(']')]
        var = var[var.find(']')+1:]
        elems = var1.split(";")
        TDegree=[{'degree': elems[0].strip(),'branch': elems[1].strip(),'specialisation': elems[2].strip(),'university': elems[3].strip(),'country': elems[4].strip(),'year': int(elems[5].strip())}]        
        while var.startswith('[') :       
            var1 = var[1:var.find(']')]
            var = var[var.find(']')+1:]
            elems = var1.split(";")
            TDegree.append({'degree': elems[0].strip(),'branch': elems[1].strip(),'specialisation': elems[2].strip(),'university': elems[3].strip(),'country': elems[4].strip(),'year': int(elems[5].strip())})  
    except ValueError as ve:
        print('value error : ', Degree)
        print('>', ve)
        
##informations relatives à des échanges Erasmus    
elems=""  
TExch=[]        
if len(Exchange_programme) > 0 :
    try:
        var=Exchange_programme
        var1 = var[1:var.find(']')]
        var = var[var.find(']')+1:]
        elems = var1.split(";")
        TExch=[{'type': elems[0].strip(), 'university': elems[1].strip(),'country': elems[2].strip(),'year': int(elems[3].strip())}]        
        while var.startswith('[') :       
            var1 = var[1:var.find(']')]
            var = var[var.find(']')+1:]
            elems = var1.split(";")
            TExch.append({'type': elems[0].strip(), 'university': elems[1].strip(),'country': elems[2].strip(),'year': int(elems[3].strip())})        
    except ValueError as ve:
        print('value error : ', Exchange_programme)
        print('>', ve)
        
##informations relatives au(x) PG(s)
elems="" 
TPG=[]
if len(PG) > 0 : 
    try:
        if PG.count(";") == 0 :
            TPG.append(PG.strip())
        else :
            TPG = PG.split(";")
            TPG.sort()  
    except ValueError as ve:
        print('value error : ', PG)
        print('>', ve)

##informations relatives au(x) expertises
elems="" 
TExpertise=[]
if len(Expertise) > 0 : 
    try:
        if Expertise.count(";") == 0 :
            TExpertise.append(Expertise.strip())
        else :
            TExpertise = Expertise.split(";")
            TExpertise.sort()  
    except ValueError as ve:
        print('value error : ', Expertise)
        print('>', ve)

print("Prénom : " + FirstName)
print("Nom : " + LastName)
print("genre : " + Gender)
print("né en : " + Birthyear)
print("PG(s) : ") ; print(TPG)
print("Expertise(s) : ") ; print(TExpertise)
print("Barreau : ") ; print(TBar)
print("Diplome : ") ; print(TDegree)
print("Echange : ") ; print(TExch)
print("Arrivée : " + Arrival)

[Master; Law; Economic Law and Social Law; Université Libre de Bruxelles; Belgium; 2015][Master; Law; Social Law; Université Libre de Bruxelles; Belgium; 2016]
Prénom : Amélie
Nom : Poulain
genre : F
né en : 1990
PG(s) : 
['Employment', 'Pensions']
Expertise(s) : 
['collective employment law', 'individual employment law', 'social security law']
Barreau : 
[{'year': 2016, 'city': 'Brussels'}]
Diplome : 
[{'degree': 'Master', 'branch': 'Law', 'specialisation': 'Economic Law and Social Law', 'university': 'Université Libre de Bruxelles', 'country': 'Belgium', 'year': 2015}, {'degree': 'Master', 'branch': 'Law', 'specialisation': 'Social Law', 'university': 'Université Libre de Bruxelles', 'country': 'Belgium', 'year': 2016}]
Echange : 
[]
Arrivée : 2019


# $TEXTE_EXPERTISE

## Planification de document

In [6]:
TEXTE_EXPERTISE=""
LIST_PROF=""

##si un PG --> TEXTE_EXPERTISE comprendra 1 phrase simple, sinon il en comprendra 2
if len(PG) > 0 : 
    if PG.count(";") == 0 :
        nbp=1
    else :
        nbp=2
        


# Version anglaise avec SimpleNLG

### Micro-planification 

In [7]:
##Lexicalisation

Verb_en="" ; Verb_en2=""
Adverb_en="" ; Adverb_en2=""
locu_en=""

##Choix aléatoire du verbe et de l'adverbe dans le cas d'une phrase (=1PG)
listeverbEN1 = ['concentrate on', 'advise clients on', 'specialise in', 'practice in']
listeadverbEN = ['in particular', 'more specifically', 'with a special focus on', 'particularly matters concerning', 'with a particular focus on']
Verb_en = random.choice(listeverbEN1)
Adverb_en = random.choice(listeadverbEN)
locu_en=""
if Adverb_en =='in particular' or Adverb_en =='more specifically' :
    if Verb_en[-3:]==' in' :
        locu_en="in"
    elif Verb_en[-3:]==' on' :
        locu_en="on"

if nbp==2 :
    ##choix des verbes dans le cas de 2 phrases (2PG)
    listeverbEN2 = ['concentrate on', 'focus on', 'advise clients on', 'specialise in', 'assist clients in', 'practice in', 'be active in']
    listeadverbEN2 = ['in particular', 'more specifically','particularly']
    Verb_en2 = random.choice(listeverbEN2)
    Adverb_en2 = random.choice(listeadverbEN2)
    while Verb_en == Verb_en2 :
        Verb_en2 = random.choice(listeverbEN2)

##Expression référencielles : choix du pronom
subj_p_C="he"
if Gender == "F" :
    subj_p_C="she"
    
##Agregation si répétition de law dans Expertise ou PG

Tvar1=[]
Tvar2=[]
TPG_en=[]
if len(TPG)>1 :
    for item in TPG :
        if item.find(" law")>=0 :
            Tvar1.append(item.replace(' law','').strip())
        else :
            Tvar2.append(item)
    if len(Tvar2)>0 :
        for item in Tvar2 :
            TPG_en.append(item)
    for item in Tvar1 :
        TPG_en.append(item)
    TPG_en[-1]=TPG_en[-1].strip() + " law"
else :
    TPG_en=TPG

Tvar1=[]
Tvar2=[]
TExpertise_en=[]
itemd=""
if len(TExpertise)>1 : 
    for item in TExpertise :
        if item.find(" law")>=0 :
            Tvar1.append(item.replace(' law','').strip())
        else :
            Tvar2.append(item)
    if len(Tvar2)>0 :
        for item in Tvar2 :
            TExpertise_en.append(item)
    for item in Tvar1 :
        TExpertise_en.append(item)
    TExpertise_en[-1]=TExpertise_en[-1].strip() + " law"
else :
    TExpertise_en=TExpertise
print(len(TExpertise_en))
        

3


### Realisation de surface en anglais

In [8]:
#Creation des objects 

lexicon = Lexicon.getDefaultLexicon()
realiser = Realiser(lexicon)
nlgFactory = NLGFactory(lexicon)

In [9]:
#creation de 3 phrases. Phrase A= phrase unique si un seul PG. Phrase B et C = phrases si plusieurs PG
p_A = nlgFactory.createClause() 
p_B = nlgFactory.createClause()
p_C = nlgFactory.createClause()

## Phrase A 
suject_NP = nlgFactory.createNounPhrase(FirstName + " " + LastName)
p_A.setSubject(suject_NP)   ## ajout du sujet     
verb_p_A = nlgFactory.createVerbPhrase(Verb_en) ## ajout du verbe 
p_A.setVerb(verb_p_A) 

p = nlgFactory.createClause()##phrase pour accueillir la phrase coordonnée

## Phrase B 
p_B.setSubject(suject_NP)   ## ajout du sujet     
p_B.setVerb(verb_p_A) ## ajout du verbe 
     
##phrase C
suject_p_C = nlgFactory.createNounPhrase(subj_p_C) ##ajout du pronom
p_C.setSubject(suject_p_C)                 
verb_p_C = nlgFactory.createVerbPhrase(Verb_en2)   
p_C.setVerb(verb_p_C)                    
p_C.addModifier("particularly")        


##ajout du ou des PGs
SN_PG= nlgFactory.createNounPhrase()
if nbp == 1 :
    for item in TPG_en :
        SN_PG= nlgFactory.createNounPhrase(item)
    
if nbp > 1 :
    if len(TPG_en) > 0 : 
        i=0
        for elem in TPG_en :
            if i==0 :
                SN_PG = nlgFactory.createNounPhrase(elem)  
            elif i==1 :
                SN_PG = nlgFactory.createCoordinatedPhrase(SN_PG,elem)    
            else :           
                SN_PG.addCoordinate(elem)  
            i+=1
    else :
        print("Erreur : pas de PG trouvé")

p_A.setObject(SN_PG)## ajout de l'objet PG
p_B.setObject(SN_PG)## ajout de l'objet PGs    

coord=""
coord=coord + (", ") + Adverb_en + " " + locu_en
coord1=nlgFactory.createNounPhrase(coord)


#ajout des expertises
SN_Exp=""  
if len(TExpertise_en) > 0 : 
    if len(TExpertise_en) == 1 :
        for elem in TExpertise_en :
            SN_Exp= nlgFactory.createNounPhrase(coord1,elem)  
    else :
        i=0
        for elem in TExpertise_en :
            if i==0 :
                SN_Exp= nlgFactory.createNounPhrase(elem)
            elif i==1 :
                SN_Exp = nlgFactory.createCoordinatedPhrase(SN_Exp,elem)    
            else :           
                SN_Exp.addCoordinate(elem)  
            i+=1

#constitution de la phrase A
#obj_p_A= nlgFactory.createNounPhrase(SN_Exp)
p=nlgFactory.createNounPhrase(coord1,SN_Exp)
p_A.addComplement(p) 
#p_A.addComplement(obj_p_A)      
output_p_A=realiser.realiseSentence(p_A) 
output_p_A=output_p_A.replace(' that','')

##construction de la phrase B
s_B = nlgFactory.createSentence(p_B)

#constitution de la phrase C
p_C.setObject(SN_Exp) 
s_C = nlgFactory.createSentence(p_C)
par = nlgFactory.createParagraph([s_B,s_C])
output_B_C = realiser.realise(par).getRealisation()
output_B_C=output_B_C.replace('particularly',Adverb_en2)

#construction du paragraphe de 2 phrases


#Selection du texte gardé en fonction du nombre de PG
if nbp==1 :
    TEXTE_EXPERTISE=output_p_A
elif nbp==2 :
    TEXTE_EXPERTISE=output_B_C
    
print(TEXTE_EXPERTISE)

Amélie Poulain specialises in Employment and Pensions law. She in particular assists clients in collective employment, individual employment and social security law.




# Version française avec CFG

## Micro planning et Realisation de surface 

In [10]:
PG_FR=""
det=""
Tmaliste1=[]
Tmaliste2=[]
ma_liste_PG = []
Exp_FR=""
ma_liste_Exp=[]
prop_adv1=""
prop=""
prop_1=""
prop_2=""
prop_Exp=""
prop_PG=""
verbe_1=""
verbe_2=""
verbe_3=""
adverbe_1=""

##Lexicalisation
##*************************

#verbe pour 1 phrase
nlg_var = NlgTools(lang='fr')
verbe_1=nlg_var.write_text(TextVar(nlg_var.nlg_syn('se concentre (sur)', 'concentre ses activités dans le domaine (de)','conseille ses clients en matière (de)','conseille ses clients sur tous les aspects relatifs (à)', 'se spécialise dans tous les aspects (de)')))
prop_1=verbe_1[verbe_1.find('(')+1:verbe_1.find(')')]
verbe_1=verbe_1[:verbe_1.find('(')].strip()

#adverbe pour 1 phrase
nlg_var = NlgTools(lang='fr')
adverbe_1=nlg_var.write_text(TextVar(nlg_var.nlg_syn('et en particulier', 'et plus particulièrement', 'et plus précisément', 'avec une attention particulière (pour)')))
##gestion de la locution adverbiale avec pour
prop_adv1=prop_1 if adverbe_1.find('(') < 0 else ''
adverbe_1=adverbe_1 if adverbe_1.find('(') < 0 else adverbe_1.replace('(','').replace(')','').strip()

#verbe pour 2 phrases - phrase 1
nlg_var = NlgTools(lang='fr')
verbe_2=nlg_var.write_text(TextVar(nlg_var.nlg_syn('se concentre (sur)', 'se concentre (sur)', 'concentre ses activités dans le domaine (de)','conseille ses clients en matière de ()','conseille ses clients sur tous les aspects relatifs (à)', 'exerce (en)', 'se spécialise dans tous les aspects (de)')))
prop_2=verbe_2[verbe_2.find('(')+1:verbe_2.find(')')]
verbe_2=verbe_2[:verbe_2.find('(')].strip()

#verbe pour 2 phrases - phrase 2
nlg_var = NlgTools(lang='fr')
verbe_3=nlg_var.write_text(TextVar(nlg_var.nlg_syn('se concentre plus précisément (sur)', 'se concentre plus particulièrement (sur)', 'concentre plus particulièrement ses activités dans le domaine (de)','conseille également ses clients en matière de ()','conseille également ses clients sur tous les aspects relatifs (à)', 'peut également se prévaloir d\'une solide expérience (dans))')))
prop_3=verbe_3[verbe_3.find('(')+1:verbe_3.find(')')]
verbe_3=verbe_3[:verbe_3.find('(')].strip()

##Expression référencielles : choix du pronom
subj_p_C="Il"
if Gender == "F" :
    subj_p_C="Elle"

    
if nbp==1 :
    prop_Exp=prop_adv1 
    prop_PG=prop_adv1
elif nbp==2 :
    prop_Exp=prop_3 
    prop_PG=prop_2    

##Agregation si répétition de droit dans PG    
if len(TPG)>1 :
    i=0
    j=0
    for item in TPG :
        PG_FR=search_trans(item.strip(),'FR')
        if PG_FR.find("droit ")>=0 :
            if i>0 :
                Tmaliste1.append(PG_FR[3:].replace('droit ',''))
            else :
                if prop_PG=="en" or prop_PG=="" :
                    PG_FR=PG_FR[3:]
                else :
                    PG_FR=PG_FR
                if j==1 :
                    prop_PG=" "                    
                Tmaliste1.append(prop_PG + " " + PG_FR)
            i+=1
        else :
            if prop_PG=="en" or prop_PG=="" :
                PG_FR=PG_FR[3:] 
            if j==1 :
                prop_PG=" " 
            Tmaliste2.append(prop_PG + " " + PG_FR)
            j=1
    if len(Tmaliste2)>0 :
        for item in Tmaliste2 :
            ma_liste_PG.append(item)
    if len(Tmaliste1)>0 :
        for item in Tmaliste1 :
            ma_liste_PG.append(item)
else :
    PG_FR=prop_PG + " " + search_trans(TPG[0].strip(),'FR')
    if prop_PG=="en" or prop_PG=="" :
        PG_FR=PG_FR[3:]
    ma_liste_PG.append(PG_FR)


##Agregation si répétition de droit dans Expertise 
Tmaliste1=[]
Tmaliste2=[]
if len(TExpertise)>1 :
    i=0
    j=0
    for item in TExpertise :
        Exp_FR=search_trans(item.strip(),'FR')
        if Exp_FR.find("droit ")>=0 :
            if i>0 :
                Tmaliste1.append((Exp_FR[3:].replace('droit ','').strip()))
            else :
                if prop_Exp=="en" or prop_Exp=="" :
                    Exp_FR=Exp_FR[3:]
                else :
                    Exp_FR=Exp_FR
                if j==1 :
                    prop_Exp=" " 
                Tmaliste1.append(prop_Exp + " " + Exp_FR.strip())
            i+=1
        else :
            if prop_Exp=="en" or prop_Exp=="" :
                Exp_FR=Exp_FR[3:]
            if j==1 :
                prop_Exp=" " 
            Tmaliste2.append(prop_Exp + " " + Exp_FR.strip())
            j=1
    if len(Tmaliste2)>0 :
        for item in Tmaliste2 :
            ma_liste_Exp.append(item)
    if len(Tmaliste1)>0 :
        for item in Tmaliste1 :
            ma_liste_Exp.append(item)
else :
    Exp_FR=prop_Exp + " " + search_trans(TExpertise[0].strip(),'FR')
    if prop_Exp=="en" or prop_Exp=="" :
        Exp_FR=Exp_FR[3:]
    ma_liste_Exp.append(Exp_FR)

nlg = NlgTools(lang='fr')

texte_1p = TextVar(
    FirstName, 
    LastName,
    verbe_1,
    prop_1,
    nlg.enum(ma_liste_PG),
    ', ',
    adverbe_1,
    nlg.enum(ma_liste_Exp),
  '.'
)

texte_2p = TextVar(
    FirstName, 
    LastName,
    verbe_2,
    nlg.enum(ma_liste_PG),
    '. ',
    subj_p_C,
    verbe_3,
    nlg.enum(ma_liste_Exp),
  '.'
)

if nbp==1:
    texte=texte_1p
else :
    texte=texte_2p
    
nlg.write_text(texte)
output=nlg.text
output=output.replace('en le','en').replace('en la','en').replace("en l'",'en')
TEXTE_EXPERTISE_FR=output

print(TEXTE_EXPERTISE_FR)


Amélie Poulain concentre ses activités dans le domaine du Droit du travail et les Pensions. Elle conseille également ses clients en matière de droit collectif du travail, individuel du travail et de la sécurité sociale.


# Version Neerlandaise avec CFG

## Micro planning et Realisation de surface 

In [11]:
PG_NL=""
det=""
Tmaliste1=[]
Tmaliste2=[]
ma_liste_PG = []
Exp_NL=""
ma_liste_Exp=[]
prop_adv1=""
prop=""
prop_1=""
prop_2=""
prop_Exp=""
prop_PG=""
verbe_1=""
verbe_2=""
verbe_3=""
adverbe_1=""

##Lexicalisation
##*************************

#verbe pour 1 ou 2 phrase - phrase 1
nlg_var = NlgTools(lang='fr')
verbe_1=nlg_var.write_text(TextVar(nlg_var.nlg_syn('concentreert zich op', 'specialiseert zich in alle aspecten van', 'specialiseert zich in','focust zich op','heeft ruime ervaring in', 'legt zich toe op', 'adviseert cliënten over alle aspecten van', 'verleent advies over', 'is gespecialiseerd in', 'geeft pragmatisch en bedrijfsgericht advies over')))

#adverbe pour 1 phrase
nlg_var = NlgTools(lang='fr')
adverbe_1=nlg_var.write_text(TextVar(nlg_var.nlg_syn(', met een bijzondere aandacht voor', ', in het bijzonder op het gebied van', ', en meer bepaald', 'met een focus op alle aspecten van')))

#verbe pour 2 phrases - phrase 2
nlg_var = NlgTools(lang='fr')
verbe_2=nlg_var.write_text(TextVar(nlg_var.nlg_syn('concentreert zich voornamelijk op', 'specialiseert zich voornamelijk in alle aspecten van', 'specialiseert zich in','focust zich voornamelijk op','heeft daarnaast ruime ervaring in', 'legt zich toe op', 'adviseert cliënten over alle aspecten van', 'verleent advies over', 'is gespecialiseerd in', 'beschikt daarnaast over een grondige kennis van', 'heeft daarnaast bijzondere ervaring op het domein van', 'is eveneens actief in')))

##Expression référencielles : choix du pronom
subj_p_C="hij"
if Gender == "F" :
    subj_p_C="ze"
    

##Agregation si répétition de droit dans PG    
if len(TPG)>1 :
    i=0
    j=0
    for item in TPG :
        PG_NL=search_trans(item.strip(),'NL')
        if TPG.count('recht')>1 :
            print("ok")
        if PG_NL.find("recht ")>=0 :
            if i>0 :
                Tmaliste1.append(PG_NL.replace('recht','-'))
            else :            
                Tmaliste1.append(PG_NL)
            i+=1
        else :
            Tmaliste2.append(PG_NL)
    if len(Tmaliste2)>0 :
        for item in Tmaliste2 :
            ma_liste_PG.append(item)
    if len(Tmaliste1)>0 :
        for item in Tmaliste1 :
            ma_liste_PG.append(item)
else :
    PG_NL=search_trans(TPG[0].strip(),'NL')
    ma_liste_PG.append(PG_NL)

##Agregation si répétition de droit dans Expertise 
Tmaliste1=[]
Tmaliste2=[]
if len(TExpertise)>1 :
    i=0
    j=0
    for item in TExpertise :
        Exp_NL=search_trans(item.strip(),'NL')
        if Exp_NL.find("recht")>=0 :
            if i>0 :
                Tmaliste1.append((Exp_NL.replace('recht','-').strip()))
            else :
                Tmaliste1.append(Exp_NL.strip())
            i+=1
        else :
            Tmaliste2.append(Exp_NL.strip())
    if len(Tmaliste2)>0 :
        for item in Tmaliste2 :
            ma_liste_Exp.append(item)
    if len(Tmaliste1)>0 :
        for item in Tmaliste1 :
            ma_liste_Exp.append(item)
else :
    Exp_NL=search_trans(TExpertise[0].strip(),'NL')
    ma_liste_Exp.append(Exp_NL)

nlg = NlgTools(lang='fr')

texte_1p = TextVar(
    FirstName, 
    LastName,
    verbe_1,
    nlg.enum(ma_liste_PG),
    ', ',
    adverbe_1,
    nlg.enum(ma_liste_Exp),
  '.'
)

texte_2p = TextVar(
    FirstName, 
    LastName,
    verbe_1,
    nlg.enum(ma_liste_PG),
    '. ',
    subj_p_C,
    verbe_2,
    nlg.enum(ma_liste_Exp),
  '.'
)

if nbp==1:
    texte=texte_1p
else :
    texte=texte_2p
    
nlg.write_text(texte)
output=nlg.text
output=output.replace(' et ', ' en ')
TEXTE_EXPERTISE_NL=output

print(TEXTE_EXPERTISE_NL)


Amélie Poulain geeft pragmatisch en bedrijfsgericht advies over het arbeidsrecht en Pensioenen. Ze focust zich voornamelijk op collectief arbeidsrecht, het individueel arbeids- en het socialezekerheids-.


# $LIST_PROF

## Réalisation de surface dans les trois langues avec SimpleNLG

In [12]:
##génération du cadre formation et carrière
##-----------------------------------------

list_p=[]
list_countries_EU = ('Suisse','Austria','Belgium','Bulgaria','Croatia','Cyprus','Czech Republic','Denmark','Estonia','Finland','France','Germany','Greece','Hungary','Ireland','Italy','Latvia','Lithuania','Luxembourg','Malta','Netherlands','Poland','Portugal','Romania','Slovakia','Slovenia','Spain','Sweden')
v_admis = "admis "
v_ne = "né en "
if Gender == "F" :
    v_admis = "admise "   ; v_ne = "née en "
    
p = nlgFactory.createClause()   ; pFR = nlgFactory.createClause()  ; pNL = nlgFactory.createClause()

obj="born in " + Birthyear     ; objFR=v_ne + Birthyear  ; objNL='Geboren in ' + Birthyear
p.setObject(obj)               ; pFR.setObject(objFR)    ; pNL.setObject(objNL)
phrase=realiser.realiseSentence(p) ; phraseFR=realiser.realiseSentence(pFR) ; phraseNL=realiser.realiseSentence(pNL)
list_p.append({'year': int(Birthyear),'phraseEN': phrase, 'phraseFR': phraseFR, 'phraseNL': phraseNL})


obj=""                                                     ; objFR=""     ; objNL=""
obj="with the Brussels office of Stibbe since " + Arrival  ; objFR="au cabinet de Stibbe Bruxelles depuis " + Arrival ; objNL="In kantoor Stibbe Brussel sinds " + Arrival
p.setObject(obj)                                           ; pFR.setObject(objFR)                                     ; pNL.setObject(objNL)
phrase=realiser.realiseSentence(p) ; phraseFR=realiser.realiseSentence(pFR) ; phraseNL=realiser.realiseSentence(pNL)
list_p.append({'year': int(Arrival),'phraseEN': phrase, 'phraseFR': phraseFR, 'phraseNL': phraseNL})

obj=""                                                     ; objFR=""     ; objNL=""
i=0
for line in TBar :
    ville=""
    ville=GoogleTranslator(source='auto', target='fr').translate(TBar[i]['city'])
    villeNL=""
    villeNL=GoogleTranslator(source='auto', target='nl').translate(TBar[i]['city'])
    obj="admitted to the " + TBar[i]['city'] + " Bar in " + str(TBar[i]['year']) ; objFR=v_admis + ("au barreau d'" if voyelle(ville[:1].lower()) else "au barreau de ") + ville  + " en " + str(TBar[i]['year'])
    objNL="Toegelaten tot de balie te " + villeNL + " in " + str(TBar[i]['year']) 
    p.setObject(obj)                                           ; pFR.setObject(objFR)                                     ; pNL.setObject(objNL)
    phrase=realiser.realiseSentence(p) ; phraseFR=realiser.realiseSentence(pFR) ; phraseNL=realiser.realiseSentence(pNL)
    list_p.append({'year': int(TBar[i]['year']),'phraseEN': phrase, 'phraseFR': phraseFR, 'phraseNL': phraseNL})
    i+=1
i=0

for line in TDegree :
    obj=""                                                     ; objFR=""         ; objNL=""
    if TDegree[i]['degree'].lower()== "ll.m." : 
        obj="Master of Laws (LL.M.)"                          ; objFR="Master (LL.M.) en droit"    ; objNL="Master of Laws (LL.M.)" 
    elif TDegree[i]['degree'].lower()== "master" : 
        obj="Master's degree in " + TDegree[i]['branch']       ; objFR="Master en " + ("droit" if TDegree[i]['branch'].lower()=="law" else GoogleTranslator(source='auto', target='fr').translate(TDegree[i]['branch']))
        objNL="Masterdiploma in " + ("de rechten" if TDegree[i]['branch'].lower()=="law" else GoogleTranslator(source='auto', target='nl').translate(TDegree[i]['branch']).lower())
    elif TDegree[i]['degree'].lower()== "bachelor" : 
        obj="Bachelor's degree in " + TDegree[i]['branch']     ; objFR="Bachelier en " + ("droit" if TDegree[i]['branch'].lower()=="law" else GoogleTranslator(source='auto', target='fr').translate(TDegree[i]['branch']))
        objNL="Bachelordiploma in " + ("de rechten" if TDegree[i]['branch'].lower()=="law" else GoogleTranslator(source='auto', target='nl').translate(TDegree[i]['branch']).lower())
    else : 
        obj=TDegree[i]['degree'] + "'s degree in " + TDegree[i]['branch'].title() ; objFR=GoogleTranslator(source='auto', target='fr').translate(TDegree[i]['degree']) + " en " + ("droit" if TDegree[i]['branch'].lower()=="law" else GoogleTranslator(source='auto', target='fr').translate(TDegree[i]['branch']))
        objNL=GoogleTranslator(source='auto', target='nl').translate(TDegree[i]['degree']) + " in " + ("de rechten" if TDegree[i]['branch'].lower()=="law" else GoogleTranslator(source='auto', target='fr').translate(TDegree[i]['branch']).lower())
    if len(TDegree[i]['specialisation'])>0 : 
        obj=obj + (" in " + TDegree[i]['specialisation'].title() if TDegree[i]['degree'].lower()== "ll.m." else ", specialisation in " + TDegree[i]['specialisation'].title() + ",")  
        elem=""
        spec=""
        spec=TDegree[i]['specialisation']
        elem=search_trans(spec,'FR')
        objFR=objFR + ((" en " + (GoogleTranslator(source='auto', target='fr').translate(TDegree[i]['specialisation']).lower() if elem=='' else elem.replace('le ','').replace('la ','').replace('l\' ','').lower())) if TDegree[i]['degree'].lower()== "ll.m." else (", spécialisation en " + (GoogleTranslator(source='auto', target='fr').translate(TDegree[i]['specialisation']).lower() if elem=='' else elem.replace('le ','').replace('la ','').replace('l\' ','').lower()) + ",")) 
        if objFR.count('en droit en droit')>0:
            objFR=objFR.replace('en droit en droit','en droit')
        elem=""
        elem=search_trans(spec,'NL')
        objNL=objNL + (" in " if TDegree[i]['degree'].lower()== "ll.m." else ", specialisatie in ") + (GoogleTranslator(source='auto', target='nl').translate(TDegree[i]['specialisation']).lower() if elem=='' else  elem.replace('de ','').replace('het ','').lower()) + ","
    obj=obj + " from the " + search_univ(TDegree[i]['university'],'EN') + ("" if TDegree[i]['country'].lower()=='belgium' else (", " + TDegree[i]['country'])) + " (" + str(TDegree[i]['year']) + ")" ; 
    objFR=objFR + " de " + search_univ(TDegree[i]['university'],'FR') + ("" if TDegree[i]['country'].lower()=='belgium' else (", " + GoogleTranslator(source='auto', target='fr').translate(TDegree[i]['country']))) + " (" + str(TDegree[i]['year']) + ")"
    objNL=objNL + " aan de " + search_univ(TDegree[i]['university'],'NL') + ("" if TDegree[i]['country'].lower()=='belgium' else (", " + GoogleTranslator(source='auto', target='nl').translate(TDegree[i]['country']))) + " (" + str(TDegree[i]['year']) + ")"
    p.setObject(obj)                                           ; pFR.setObject(objFR)                                     ; pNL.setObject(objNL)
    phrase=realiser.realiseSentence(p) ; phraseFR=realiser.realiseSentence(pFR) ; phraseNL=realiser.realiseSentence(pNL)
    list_p.append({'year': int(TDegree[i]['year']),'phraseEN': phrase, 'phraseFR': phraseFR, 'phraseNL': phraseNL})
    i+=1
i=0
if len(TExch)>0 :
    for line in TExch :
        obj=""                         ; objFR=""   ; objNL=""
        if TExch[i]['country'] in list_countries_EU : 
            obj="Erasmus programme "   ; objFR="programme Erasmus "   ; objNL="Erasmus programma " 
        else : 
            obj="Exchange programme "  ; objFR="programme d'échange " ; objNL="Uitwisselingsprogramma " 
        obj=obj + "at the " + search_univ(TExch[i]['university'],'EN') + ("" if TExch[i]['country'].lower()=='belgium' else (", " + TExch[i]['country'])) + " (" + str(TExch[i]['year']) + ")"
        objFR=objFR + "à " + search_univ(TExch[i]['university'],'FR') + ("" if TExch[i]['country'].lower()=='belgium' else (", " + GoogleTranslator(source='auto', target='fr').translate(TExch[i]['country']))) + " (" + str(TExch[i]['year']) + ")"
        objNL=objNL + "aan de " + search_univ(TExch[i]['university'],'NL') + ("" if TExch[i]['country'].lower()=='belgium' else (", " + GoogleTranslator(source='auto', target='nl').translate(TExch[i]['country']))) + " (" + str(TExch[i]['year']) + ")"
        p.setObject(obj)                                           ; pFR.setObject(objFR)                                     ; pNL.setObject(objNL)
        phrase=realiser.realiseSentence(p) ; phraseFR=realiser.realiseSentence(pFR) ; phraseNL=realiser.realiseSentence(pNL)
        list_p.append({'year': int(TExch[i]['year']),'phraseEN': phrase, 'phraseFR': phraseFR, 'phraseNL': phraseNL})
        i+=1


##production du texte
##---------------------

list_p.sort(key=sortbyyear, reverse=False)

print("reference : ")
print("**********************************************************")
print("Education, career and qualifications")
for item in list_p :
    print((item['phraseEN'])[:-1])
print()
print("Formation et carrière")
for item in list_p :
    print((item['phraseFR'])[:-1])
print()
print("Opleiding en carrière")
for item in list_p :
    print((item['phraseNL'])[:-1])


Pas de traduction pour Economic Law and Social Law
Pas de traduction pour Economic Law and Social Law
Pas de traduction pour Social Law
Pas de traduction pour Social Law
reference : 
**********************************************************
Education, career and qualifications
Born in 1990
Master's degree in Law, specialisation in Economic Law And Social Law, from the <i>Université libre de Bruxelles</i> (2015)
Admitted to the Brussels Bar in 2016
Master's degree in Law, specialisation in Social Law, from the <i>Université libre de Bruxelles</i> (2016)
With the Brussels office of Stibbe since 2019

Formation et carrière
Née en 1990
Master en droit, spécialisation en droit économique et droit social, de l'Université libre de Bruxelles (2015)
Admise au barreau de Bruxelles en 2016
Master en droit, spécialisation en droit social, de l'Université libre de Bruxelles (2016)
Au cabinet de Stibbe Bruxelles depuis 2019

Opleiding en carrière
Geboren in 1990
Masterdiploma in de rechten, special

#Ecriture des résultats dans le fichier resultat.txt

In [13]:
filename="resultat/"+pers+"pipeline.txt" 
with open(filename, 'w') as file:
    file.write(FirstName.upper() + " " + LastName.upper())
    file.write("\n")
    file.write("English :")
    file.write("\n")
    file.write(TEXTE_EXPERTISE)
    file.write("Education, career and qualifications")
    file.write("\n")
    for item in list_p :
        file.write(("> " + item['phraseEN'])[:-1])
        file.write("\n")
    file.write("\n")
    file.write("\n")
    file.write("Français :")
    file.write("\n")
    file.write(TEXTE_EXPERTISE_FR) 
    file.write("Formation et carrière")
    file.write("\n")
    for item in list_p :
        file.write(("> " + item['phraseFR'])[:-1])
        file.write("\n")
    file.write("\n")
    file.write("\n")
    file.write("Nederlands :")
    file.write("\n")
    file.write(TEXTE_EXPERTISE_NL) 
    file.write("Opleiding en carrière")
    file.write("\n")
    for item in list_p :
        file.write(("> " + item['phraseNL'])[:-1])    
        file.write("\n")
file.close()